In [1]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [2]:
base_path = os.getcwd()

In [3]:
# Load script from specified path sys.argv[1] (snakemake {input})
# Does nothing else than `import path/to/script as exp` would do
experiment_file = './experiments/experiment_stabilized_groundstate.py'
conf_name, _ = os.path.splitext(os.path.basename(experiment_file))
conf_path = os.path.join(os.getcwd(), './experiments/experiment_stabilized_groundstate.py')
spec = importlib.util.spec_from_file_location(conf_name, conf_path)
exp = importlib.util.module_from_spec(spec)
spec.loader.exec_module(exp)

In [4]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
outpath = net_params['outpath']

net_params['N_scaling'] = 0.001
net_params['K_scaling'] = 0.001

In [5]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

Dropping layer IV in caudalmiddlefrontal because it contains 3375 < 5000 neurons.
Dropping layer IV in lateraloccipital because it contains 3000 < 5000 neurons.
Dropping layer VI in parahippocampal because it contains 3200 < 5000 neurons.
Dropping layer IV in parsorbitalis because it contains 625 < 5000 neurons.


In [6]:
# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

In [7]:
net = Network(NN, SN, net_params)

In [8]:
# Get the network hash
net_hash = net.getHash()

In [9]:
net.net.keys()

dict_keys(['area_list', 'layer_list', 'population_list', 'neuron_model_E', 'neuron_model_I', 'neuron_params_E', 'neuron_params_I', 'neuron_param_dist_E', 'neuron_param_dist_I', 'neuron_numbers', 'SLN', 'synapses_internal', 'synapses_external', 'weights', 'weights_ext', 'p_transmit', 'delay_e', 'delay_e_sd', 'delay_i', 'delay_i_sd', 'delay_cc', 'delay_cc_sd', 'delay_distribution', 'eta_ext', 'rate_ext', 'spike_time', 'weights_sd', 'weights_ext_sd', 'NOS', 'directionality'])

In [11]:
net.net['neuron_numbers'].sum()

3468.839